In [1]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy

In [2]:
def show_image(title,image):
    image=cv.resize(image,(0,0),fx=0.3,fy=0.3)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

def show_image_plt(title, image):
    plt.figure(figsize=(image.shape[1] * 0.003, image.shape[0] * 0.003))
    
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')
    plt.show()

In [3]:
BOARD_WIDTH = 14 # width size of the table
BOARD_HEIGHT = 14 # height size of the table
BOARD_WIDTH_PX = 1975 # width of the big table in pixels
BOARD_HEIGHT_PX = 1975 # height of the big table in pixels
BOARD_BORDER_PX = 250 # size of the border needed to be removed in order to get the small table (where the pieces are placed)

SMALL_BOARD_WIDTH_PX = 14 * 104 # 1713 - 257
SMALL_BOARD_HEIGHT_PX = 14 * 104 # 1713 - 257
TILE_SPACE = SMALL_BOARD_WIDTH_PX // BOARD_WIDTH

In [4]:
# this data class stores the information of a tile
class Tile:
    def __init__(self, i, j, value = None, bonus = 1, condition = None):
        self.i = i
        self.j = j
        self.value = value # value of the piece placed on the tile
        self.bonus = bonus # bonus of the tile - 1, 2 or 3
        self.condition = condition # condition of the tile - None, +, -, x, /

In [5]:
board = np.array([[Tile(i, j) for j in range(BOARD_WIDTH)] for i in range(BOARD_HEIGHT)])

# first 4 pieces
board[6][6] = Tile(6,6,1,1); board[6][7] = Tile(6,7,2,1); board[7][6] = Tile(7,6,3,1); board[7][7] = Tile(7,7,4,1)

# conditioned pieces
board[1][4] = Tile(1,4,None,1,"/"); board[1][9] = Tile(1,9,None,1,"/")
board[2][5] = Tile(2,5,None,1,"-"); board[2][8] = Tile(2,8,None,1,"-")
board[3][6] = Tile(3,6,None,1,"+"); board[3][7] = Tile(3,7,None,1,"x")
board[4][1] = Tile(4,1,None,1,"/"); board[4][6] = Tile(4,6,None,1,"x"); board[4][7] = Tile(4,7,None,1,"+"); board[4][12] = Tile(4,12,None,1,"/")
board[5][2] = Tile(5,2,None,1,"-"); board[5][11] = Tile(5,11,None,1,"-")
board[6][3] = Tile(6,3,None,1,"x"); board[6][4] = Tile(6,4,None,1,"+"); board[6][9] = Tile(6,9,None,1,"x"); board[6][10] = Tile(6,10,None,1,"+")
board[7][3] = Tile(7,3,None,1,"+"); board[7][4] = Tile(7,4,None,1,"x"); board[7][9] = Tile(7,9,None,1,"+"); board[7][10] = Tile(7,10,None,1,"x")
board[8][2] = Tile(8,2,None,1,"-"); board[8][11] = Tile(8,11,None,1,"-")
board[9][1] = Tile(9,1,None,1,"/"); board[9][6] = Tile(9,6,None,1,"+"); board[9][7] = Tile(9,7,None,1,"x"); board[9][12] = Tile(9,12,None,1,"/")
board[10][6] = Tile(10,6,None,1,"x"); board[10][7] = Tile(10,7,None,1,"+")
board[11][5] = Tile(11,5,None,1,"-"); board[11][8] = Tile(11,8,None,1,"-")
board[12][4] = Tile(12,4,None,1,"/"); board[12][9] = Tile(12,9,None,1,"/")

# 2x bonus
board[1][1] = Tile(1,1,None,2); board[2][2] = Tile(2,2,None,2); board[3][3] = Tile(3,3,None,2); board[4][4] = Tile(4,4,None,2)
board[12][1] = Tile(12,1,None,2); board[11][2] = Tile(11,2,None,2); board[10][3] = Tile(10,3,None,2); board[9][4] = Tile(9,4,None,2)
board[1][12] = Tile(1,12,None,2); board[2][11] = Tile(2,11,None,2); board[3][10] = Tile(3,10,None,2); board[4][9] = Tile(4,9,None,2)
board[9][9] = Tile(9,9,None,2); board[10][10] = Tile(10,10,None,2); board[11][11] = Tile(11,11,None,2); board[12][12] = Tile(12,12,None,2)

# 3x bonus
board[0][0] = Tile(0,0,None,3); board[0][6] = Tile(0,6,None,3); board[0][7] = Tile(0,7,None,3); board[0][13] = Tile(0,13,None,3)
board[6][0] = Tile(6,0,None,3); board[7][0] = Tile(7,0,None,3); board[6][13] = Tile(6,13,None,3); board[7][13] = Tile(7,13,None,3)
board[13][0] = Tile(13,0,None,3); board[13][6] = Tile(13,6,None,3); board[13][7] = Tile(13,7,None,3); board[13][13] = Tile(13,13,None,3)

available_pieces = []
for i in range(0, 22):
    available_pieces.append(i)
available_pieces.extend([24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 50, 54, 56, 60, 63, 64, 70, 72, 80, 81, 90])

In [6]:
def get_board_image(image):
    # this function gets the small board from the image
    hsv_image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    
    low = np.array([80, 90, 0])
    high = np.array([120, 255, 255])
    mask = cv.inRange(hsv_image, low, high) # mask using hsv changes
        
    mask_median_blur = cv.medianBlur(mask, 3)
    mask_gaussian_blur = cv.GaussianBlur(mask_median_blur, (0, 0), 5)
    mask_sharpened = cv.addWeighted(mask_median_blur, 1.2, mask_gaussian_blur, -0.8, 0)
    
    _, mask_threshold = cv.threshold(mask_sharpened, 30, 255, cv.THRESH_BINARY)
    kernel = np.ones((3, 3), np.uint8)
    mask_threshold = cv.erode(mask_threshold, kernel)

    edges =  cv.Canny(mask_threshold ,200,400)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
        
    for i in range(len(contours)):
        if len(contours[i]) > 3:
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1) # diff between x and y coordinates
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area: # save max area contour
                # for the big table
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left
                
    image_copy = image.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
#    show_image_plt("detected corners",cv.cvtColor(image_copy, cv.COLOR_BGR2RGB))
    
#    print("board corners: ", top_left, top_right, bottom_left, bottom_right)
        
    puzzle = np.array([top_left, top_right, bottom_left, bottom_right], dtype='float32')
    destination = np.array([[0, 0], [BOARD_WIDTH_PX-1, 0], [0, BOARD_HEIGHT_PX-1], [BOARD_WIDTH_PX-1, BOARD_HEIGHT_PX-1]], dtype='float32')
    M = cv.getPerspectiveTransform(puzzle, destination) # matrix for perspective transform
    result = cv.warpPerspective(image, M, (BOARD_WIDTH_PX, BOARD_HEIGHT_PX))    
    
    cropped_result = result[BOARD_BORDER_PX:BOARD_HEIGHT_PX - BOARD_BORDER_PX + 1, BOARD_BORDER_PX:BOARD_WIDTH_PX - BOARD_BORDER_PX + 1]
    #cropped_result = cv.cvtColor(cropped_result, cv.COLOR_GRAY2BGR)
    
    # fixed corners: 257, 257 top left 
    # 1712/1715, 1722 bottom right -> 1713 makes the difference divisible by 14
    
    small_top_left = [257, 257]
    small_bottom_right = [1713, 1713]
    small_top_right = [1713, 257]
    small_bottom_left = [257, 1713]
    
    small_puzzle = np.array([small_top_left, small_top_right, small_bottom_left, small_bottom_right], dtype='float32')
    small_destination = np.array([[0, 0], [SMALL_BOARD_WIDTH_PX-1, 0], [0, SMALL_BOARD_HEIGHT_PX-1], [SMALL_BOARD_WIDTH_PX-1, SMALL_BOARD_HEIGHT_PX-1]], dtype='float32')
    small_M = cv.getPerspectiveTransform(small_puzzle, small_destination) # matrix for perspective transform
    small_result = cv.warpPerspective(result, small_M, (SMALL_BOARD_WIDTH_PX, SMALL_BOARD_HEIGHT_PX))
    
    return small_result

In [7]:
def get_lines():
    lines_horizontal = []
    lines_vertical = []
    for i in range(0, SMALL_BOARD_HEIGHT_PX + TILE_SPACE, TILE_SPACE):
        line = []
        line.append([0, i])
        line.append([SMALL_BOARD_WIDTH_PX, i])
        lines_horizontal.append(line)
    for i in range(0, SMALL_BOARD_WIDTH_PX + TILE_SPACE, TILE_SPACE):
        line = []
        line.append([i, 0])
        line.append([i, SMALL_BOARD_HEIGHT_PX])
        lines_vertical.append(line)
        
    return lines_horizontal, lines_vertical

lines_horizontal, lines_vertical = get_lines()

In [8]:
def classify_piece(patch):
    maxim = -np.inf
    pos = -1
    correlation_values = []
    
    for j in range(len(available_pieces)):
        img_template = cv.imread('templates/'+str(available_pieces[j])+'.jpg')
        grey_template = cv.cvtColor(img_template, cv.COLOR_BGR2GRAY)
        #img_template_threshold = cv.threshold(grey_template, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
        _, img_template_threshold = cv.threshold(grey_template, 65, 255, cv.THRESH_BINARY)
        
        img_template_threshold = img_template_threshold[15:img_template_threshold.shape[0]-15, 7:img_template_threshold.shape[1]-7]
        

        corr = cv.matchTemplate(patch, img_template_threshold, cv.TM_CCOEFF_NORMED)
        corr = np.max(corr)
        correlation_values.append(corr)
                
        if corr > maxim:
            maxim = corr
            pos = j
    return available_pieces[pos], maxim, correlation_values

In [9]:
empty_board = copy.deepcopy(board)

def encode_position(i, j):
    return str(str((i+1)) + chr((j + ord('A'))))

def decode_position(position):
    return int(position[:-1]) - 1, ord(position[-1:]) - ord('A')

def decode_turn(turn_file):
    with open(turn_file, 'r') as f:
        line = f.readlines()[0]
        line = line.split()
        position = decode_position(line[0])
        piece = int(line[1])
        return position, piece
    
def sort_key(file_name):
    base_name, _ = os.path.splitext(file_name)
    if base_name.split('_')[1] == "turns":
        return int(base_name.split('_')[0]), -2
    if base_name.split('_')[1] == "scores":
        return int(base_name.split('_')[0]), -1
    return int(base_name.split('_')[0]), int(base_name.split('_')[1])

def find_added_piece_on_board_matrix(previous_board, current_board):
    for i in range(BOARD_HEIGHT):
        for j in range(BOARD_WIDTH):
            if previous_board[i][j].value is None and current_board[i][j].value is not None:
                return i, j, current_board[i][j].value
    return None

In [10]:
def find_difference_between_images_using_diff_first(board1, board2, board1_matrix, verbose = False):
    # we have the images of the two boards
    # we will find the difference between the two boards
    # and identify the row and column where the difference is the biggest
    # we will return the position of the new piece and the value of the new piece
    gray_board1 = cv.cvtColor(board1, cv.COLOR_BGR2GRAY)
    gray_board2 = cv.cvtColor(board2, cv.COLOR_BGR2GRAY)
    
    _, gray_board1_thresh = cv.threshold(gray_board1, 35, 255, cv.THRESH_BINARY)
    _, gray_board2_thresh = cv.threshold(gray_board2, 35, 255, cv.THRESH_BINARY)
    
    diff = abs(gray_board1_thresh - gray_board2_thresh)
    diff = diff.astype(np.int32) # so we don't have problems when adding
    
    max_diff = -1
    diff_tile = None
    diff_mat = np.zeros((BOARD_HEIGHT, BOARD_WIDTH))
    
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0] + 20
            y_max = lines_vertical[j + 1][1][0] - 20
            x_min = lines_horizontal[i][0][1] + 20
            x_max = lines_horizontal[i + 1][1][1] - 20
            current_diff = np.sum(diff[x_min:x_max, y_min:y_max])
            diff_mat[i][j] = current_diff
            
            if current_diff > max_diff and board1_matrix[i][j].value is None:
                max_diff = current_diff
                diff_tile = [i, j]
                
    tile_patch_y_min = max(lines_vertical[diff_tile[1]][0][0] + 3, 0) 
    tile_patch_y_max = min(lines_vertical[diff_tile[1] + 1][1][0] - 3, SMALL_BOARD_WIDTH_PX) 
    tile_patch_x_min = max(lines_horizontal[diff_tile[0]][0][1] + 3, 0)
    tile_patch_x_max = min(lines_horizontal[diff_tile[0] + 1][1][1] - 3, SMALL_BOARD_HEIGHT_PX)
    #tile_patch = copy.deepcopy(gray_board2[tile_patch_x_min:tile_patch_x_max, tile_patch_y_min:tile_patch_y_max])
    
    #gray_otsu_board2 = cv.threshold(gray_board2, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
    _, gray_otsu_board2 = cv.threshold(gray_board2, 65, 255, cv.THRESH_BINARY)
    tile_patch = copy.deepcopy(gray_otsu_board2[tile_patch_x_min:tile_patch_x_max, tile_patch_y_min:tile_patch_y_max])
    # extend the tile patch with white around it
    tile_patch = cv.copyMakeBorder(tile_patch, 30, 30, 30, 30, cv.BORDER_CONSTANT, value = 255)
    
    piece_value, corr_value, corr_values = classify_piece(tile_patch)
    
    return diff_tile, piece_value, diff_mat, corr_value, corr_values

In [11]:
def get_score_for_piece(board_matrix, i, j, piece):
    score = 0
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    for direction in directions:
        neighbour_1 = [i + direction[0], j + direction[1]]
        neighbour_2 = [i + 2 * direction[0], j + 2 * direction[1]]
        if neighbour_1[0] >= 0 and neighbour_1[0] < BOARD_HEIGHT and neighbour_1[1] >= 0 and neighbour_1[1] < BOARD_WIDTH and neighbour_2[0] >= 0 and neighbour_2[0] < BOARD_HEIGHT and neighbour_2[1] >= 0 and neighbour_2[1] < BOARD_WIDTH:
            neighbour_1_piece = board_matrix[neighbour_1[0]][neighbour_1[1]].value
            neighbour_2_piece = board_matrix[neighbour_2[0]][neighbour_2[1]].value
            
            if neighbour_1_piece is not None and neighbour_2_piece is not None:
                
                if board_matrix[i][j].condition == None:
                    if neighbour_1_piece + neighbour_2_piece == piece:
                        score += piece
                    elif abs(neighbour_1_piece - neighbour_2_piece) == piece:
                        score += piece
                    elif neighbour_1_piece * neighbour_2_piece == piece:
                        score += piece
                    elif min(neighbour_1_piece, neighbour_2_piece) != 0 and max(neighbour_1_piece, neighbour_2_piece) % min(neighbour_1_piece, neighbour_2_piece) == 0 and max(neighbour_1_piece, neighbour_2_piece) // min(neighbour_1_piece, neighbour_2_piece) == piece:
                        score += piece
                else:
                    if board_matrix[i][j].condition == "+":
                        if neighbour_1_piece + neighbour_2_piece == piece:
                            score += piece
                    elif board_matrix[i][j].condition == "-":
                        if abs(neighbour_1_piece - neighbour_2_piece) == piece:
                            score += piece
                    elif board_matrix[i][j].condition == "x":
                        if neighbour_1_piece * neighbour_2_piece == piece:
                            score += piece
                    elif board_matrix[i][j].condition == "/":
                        if min(neighbour_1_piece, neighbour_2_piece) != 0 and max(neighbour_1_piece, neighbour_2_piece) % min(neighbour_1_piece, neighbour_2_piece) == 0 and max(neighbour_1_piece, neighbour_2_piece) // min(neighbour_1_piece, neighbour_2_piece) == piece:
                            score += piece

    return board_matrix[i][j].bonus * score           

In [14]:
# we will make the difference between the two images and then we will find the row and column where the difference is the biggest
# then make template matching on that area
# finally, compute the scores

working_dir = "evaluare/fake_test"
files = os.listdir(working_dir)
files = sorted(files, key = sort_key) # sort the files
empty_board_image = get_board_image(cv.imread("imagini_auxiliare/01.jpg")) # the first image is the empty board
previous_board_image = copy.deepcopy(empty_board_image) # we start with an empty board
previous_board_matrix = copy.deepcopy(empty_board)

score_obtained_per_move = np.zeros((250, 250), dtype = np.int32)
os.mkdir("351_Ciuperceanu_Vlad")

for i in tqdm(range(len(files))):
    file = files[i]
    if file[-3:] == "jpg":
        img = cv.imread(working_dir+"/"+file)
        board_image = get_board_image(img)
        
        if file[-6:-4] == "01":
            previous_board_image = copy.deepcopy(empty_board_image)
            previous_board_matrix = copy.deepcopy(empty_board)
            
        try:
            new_piece_info = find_difference_between_images_using_diff_first(previous_board_image, board_image, previous_board_matrix, verbose = False)
    
            if new_piece_info is not None:
                new_piece_ij, new_piece, diff_mat, corr_value, corr_values = new_piece_info
                new_piece_i, new_piece_j = new_piece_ij
            else:
                print("Error at file: ", file)
                print("No piece added")
                continue
                
            game = int(file.split('_')[0])
            move = int(file[:-4].split('_')[1])
                
            # write the piece info
            move_file = "351_Ciuperceanu_Vlad/" + file[:-4] + ".txt"
            with open(move_file, 'w') as f:
                f.write(encode_position(new_piece_i, new_piece_j) + " " + str(new_piece))
                
            # get the score    
            score_obtained = get_score_for_piece(previous_board_matrix, new_piece_i, new_piece_j, new_piece)
            score_obtained_per_move[game][move] = score_obtained
            
            previous_board_image = copy.deepcopy(board_image)
            previous_board_matrix[new_piece_i][new_piece_j].value = new_piece
        except Exception as e:
            print("Error at file: ", file)
            previous_board_image = copy.deepcopy(board_image)
            #previous_board_matrix[new_piece_i][new_piece_j].value = new_piece
        

# now write the scores for the player rounds
for i in range(len(files)):
    file = files[i]
    if file[-9:] == "turns.txt":
        game = file.split('_')[0]
        moves = []
        
        try:
            with open(working_dir+"/"+file, 'r') as f:
                lines = f.readlines()
                
                # copy the file in the new directory
                with open("351_Ciuperceanu_Vlad/"+file, 'w') as f2:
                    for line in lines:
                        f2.write(line)
    
                for line in lines:
                    split_line = line.split()
                    player = int(split_line[0][-1])
                    move = int(split_line[1])
                    moves.append((player, move))
                    
                prev_move = 1
                prev_player = moves[0][0]
                player_scores = np.zeros((10, 100), dtype = np.int32)
                for j in range(1, len(moves)):
                    current_player, current_move = moves[j]
                    
                    player_scores[prev_player][prev_move] = np.sum(score_obtained_per_move[int(game)][prev_move:current_move])
                    
                    prev_move = current_move
                    prev_player = current_player
        
                prev_player = current_player
                player_scores[prev_player][prev_move] = np.sum(score_obtained_per_move[int(game)][prev_move:51])
                
                scores_file = "351_Ciuperceanu_Vlad/" + game + "_scores.txt"
                with open(scores_file, 'w') as f:
                    for j in range(len(lines)):
                        split_line = lines[j].split()
                        player = int(split_line[0][-1])
                        move = int(split_line[1])
                        
                        if j == len(lines) - 1:
                            f.write(split_line[0] + " " + split_line[1] + " " + str(player_scores[player][move]))
                        else:
                            f.write(split_line[0] + " " + split_line[1] + " " + str(player_scores[player][move]) + "\n")
        except Exception as e:
            print("Error at file: ", file)
            print(e)


100%|██████████| 102/102 [00:28<00:00,  3.53it/s]
